In [ ]:
#Object Tracking
import cv2
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt

#-------- functions

def get_angle(points):
    p1,p2 = points
    p1_produto_escalar_p1 =  (p1[0]*p2[0]) + (p1[1]*p2[1])
    modulo_p1 = math.sqrt((p1[0]*p1[0]) + (p1[1]*p1[1]))
    modulo_p2 = math.sqrt((p2[0]*p2[0]) + (p2[1]*p2[1]))
    produto = modulo_p1*modulo_p2
    angle = math.acos(p1_produto_escalar_p1/produto)
    #angle = math.atan(p2[1]-p1[1]/p2[0]-p1[0])*(180/3.14)
    return math.degrees(angle)


#------------------
# Initalize camera
cap = cv2.VideoCapture(0)
# define range of purple color in HSV
lower_purple = np.array([0,108, 126])
upper_purple = np.array([8,255,255])
# globals variables
text = "Posicione sua mao dentro do quadrado - FINGER TAPPING - IFAL 2017"
# Create empty points array
points = []
y_axes  = []
x_axes = []
now = 0

# Get default camera window size
ret, frame = cap.read()
Height, Width = frame.shape[:2]
h_center = int(Height/2)
w_center= int(Width/2)
point_left_top_rec = (w_center-150, h_center-150)
point_bottom_right_rec = (w_center+150, h_center+150)
frame_count = 0
now = datetime.now()
while True:
    
    # Capture webcame frame
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
   
    # Threshold the HSV image to get only pink colors
    mask = cv2.inRange(hsv_img, lower_purple, upper_purple)
 
    #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    Gaussian = cv2.GaussianBlur(mask, (3,3), 0)
    # Find contours, OpenCV 3.X users use this line instead
    #  _, contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    _,contours, _ = cv2.findContours(Gaussian.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #organizando os contornos de maior area para menor
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    # Create empty centre array to store centroid center of mass
    center =   int(Height/2), int(Width/2)

    if len(contours) > 0:
      
        # Get the largest contour and its center 
        #c = max(contours, key=cv2.contourArea)
        #iterando sobre cada cotorno
        for c in contours:  
            (x, y), radius = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            try:
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            except:
                center =   int(Height/2), int(Width/2)
            # Allow only countors that have a larger than 15 pixel radius
            if radius > 25:
                # Draw cirlce and leave the last center creating a trail
                cv2.circle(frame, (int(x), int(y)), int(radius),(0, 0, 255), 2)
                cv2.circle(frame, center, 5, (0, 255, 0), -1)
                cv2.line(frame, (Width,0), center, (255,127,0), 5)
            # Log center points 
            points.append(center)
            
   
            
    # Display our object tracker
    
    frame = cv2.flip(frame, 1)   
    cv2.rectangle(frame, point_left_top_rec, point_bottom_right_rec, (127,50,127), 2)
    cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    if len(points) == 2:    
        angle = get_angle(points)
        cv2.putText(frame, "Angulo:"+str(angle), (70,80), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 1)   
        y_axes.append(int(angle))
       
        
        
   
    cv2.imshow("Object Tracker", frame)
    cv2.imshow("mask", mask)
    cv2.imshow("gaussian blur", Gaussian)  
    points = []
   
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

#-------plot graphs
    
fig, ax = plt.subplots()
ax.plot(y_axes)
plt.show()
    
    
#-----------------
            
          
        
# Release camera and close any open windows
cap.release()
cv2.destroyAllWindows()